In [1]:
import psycopg2 as pg
import pandas as pd
from queries.db_queries import *
from functions.running_avg import *
import numpy as np
from numpy.linalg import lstsq
import statsmodels.api as sm
#import libraries.libraries

In [2]:
# Retrosheet contains data from teams who have moved/renamed. Therefore, some abbreviations differ from Lahman DB. This dict maps them
renamed_team = {
    'NYN': 'NYM',
    'CHA': 'CHI',
    'SFN': 'SFG',
    'ANA': 'LAA',
    'CHN': 'CHC',
    'TBA': 'TBR',
    'LAN': 'LAD',
    'SDN': 'SDP',
    'WAS': 'WSN',
    'SLN': 'STL',
    'KCA': 'KCR',
    'NYA': 'NYY',
}

In [3]:
game_log_column_headers = [
    "Date",
    "Game Number",
    "Day of Week",
    "Visiting Team",
    "Visiting Team League",
    "Visiting Team Game Number",
    "Home Team",
    "Home Team League",
    "Home Team Game Number",
    "Visiting Team Score",
    "Home Team Score",
    "Length of Game (Outs)",
    "Day/Night Indicator",
    "Park ID",
    "Attendance",
    "Time of Game"
]

In [4]:
# Step 1 : Call Funciton to Return all Parks
# Step 2 : Loop through each park calculating metrics for each
# Step 3 : Devise way to normalize
# Step 4 : Compare to Park Factors from Lahman
# Step 5 :
# Step 6 : Use New metric to predict player performance
team_data = sql_query(curr_teams_query)

## Get Game Logs and Rename Abbr.

In [5]:
log_path_2023 = 'C:/Users/conor/Downloads/gl2023/gl2023.txt'

gamelogs = pd.read_csv(log_path_2023, header=None)
gamelogs_ = pd.concat([gamelogs.iloc[:,:13], gamelogs.iloc[:,16:19]], axis = 1)
gamelogs_ = gamelogs_.set_axis(game_log_column_headers, axis = 1)

#Fix Abbreviations
gamelogs_['Visiting Team'] = gamelogs_['Visiting Team'].replace(renamed_team)
gamelogs_['Home Team'] = gamelogs_['Home Team'].replace(renamed_team)

In [21]:
Phils_Games = gamelogs_[(gamelogs_['Home Team'] == 'PHI') |(gamelogs_['Visiting Team'] == 'PHI') ]
Opp_Score = []

running_avg_df = get_running_avg_df(gamelogs_=gamelogs_, timeframe=15)

for idx, game in gamelogs_.iterrows():
    
    if game['Home Team'] == 'PHI':
        opponent = game['Visiting Team']
        score = 'VTRA_Norm'
        homeaway = 'Visiting Team'
    else:
        opponent = game['Home Team']
        score = 'HTRA_Norm'
        homeaway = 'Home Team'    
    date = game.Date
    opp_score = running_avg_df[(running_avg_df['Date']== date) & (running_avg_df[homeaway]== opponent)][score].values
    Opp_Score.append(np.mean(opp_score) if len(opp_score) > 0 else None)
gamelogs_['Game Difficulty'] = Opp_Score


In [22]:
gamelogs_.reset_index(drop=True, inplace=True)
gamelogs_.head()

,Date,Game Number,Day of Week,Visiting Team,Visiting Team League,Visiting Team Game Number,Home Team,Home Team League,Home Team Game Number,Visiting Team Score,Home Team Score,Length of Game (Outs),Day/Night Indicator,Park ID,Attendance,Time of Game,Game Difficulty
0,20230330,0,Thu,MIL,NL,1,CHC,NL,1,0,4,51,D,CHI11,36054.0,141,0.296296
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,4,54,D,CIN09,44063.0,182,0.296296
2,20230330,0,Thu,ARI,NL,1,LAD,NL,1,2,8,51,N,LOS03,52075.0,155,0.592593
3,20230330,0,Thu,NYM,NL,1,MIA,NL,1,5,3,54,D,MIA02,31397.0,162,0.222222
4,20230330,0,Thu,COL,NL,1,SDP,NL,1,7,2,54,N,SAN02,45103.0,176,0.148148


In [23]:
num_feats = 3
dummy_var = len(pd.unique(gamelogs_[['Home Team', 'Visiting Team']].values.ravel()))
A = np.zeros((162*30,dummy_var*num_feats))
b = np.zeros((162*30,1))


In [24]:
team_data = team_data.sort_values(by ='Team')
team_data.reset_index(drop=True, inplace=True)

In [25]:
team_data.head()

,Team,Name,Wins,Losses,Winning %,3Yr B PF,3Yr P PF,yearid
0,ARI,Arizona Diamondbacks,84,78,0.519,99,99,2023
1,ATL,Atlanta Braves,104,58,0.642,103,101,2023
2,BAL,Baltimore Orioles,101,61,0.623,96,96,2023
3,BOS,Boston Red Sox,78,84,0.481,106,106,2023
4,CHC,Chicago Cubs,83,79,0.512,103,102,2023


In [41]:
idx = 0
weights = []
# Group by 'Home Team' and 'Visiting Team' to calculate the occurrence of each matchup
matchup_counts = gamelogs_.groupby(['Home Team', 'Visiting Team']).size().reset_index(name='Count')

# Iterate over the grouped matchups
for (home_team, visiting_team), group in gamelogs_.groupby(['Home Team', 'Visiting Team']):
    home_team_idx = team_data.index[team_data['Team'] == home_team]
    visiting_team_idx = team_data.index[team_data['Team'] == visiting_team]
    
    for _, row in group.iterrows():
        # Set offensive team factor
        A[idx, home_team_idx * num_feats] = 1
        A[idx + 1, visiting_team_idx * num_feats] = 1

        # Set defensive team factor
        A[idx, visiting_team_idx * num_feats + 1] = 1
        A[idx + 1, home_team_idx * num_feats + 1] = 1

        # Set park factor (assuming a specific column index for park factors)
        A[idx, home_team_idx * num_feats + 2] = 1  # Adjust this index if park factors are in a different column
        A[idx + 1, home_team_idx * num_feats + 2] = 1

        b[idx] = row['Home Team Score']
        b[idx + 1] = row['Visiting Team Score']
        
        idx += 2
        
        weights.append(len(group))
        weights.append(len(group))

weights = np.array(weights)

In [26]:
# Each game has two rows. First one when the home team perspective, second one for the visiting
idx = 0
for index, row in gamelogs_.iterrows():
    home_team_idx = team_data.index[team_data['Team'] == row['Home Team']] 
    visiting_team_idx = team_data.index[team_data['Team'] == row['Visiting Team']] 

    # Set offensive team factor
    A[idx, (home_team_idx*num_feats)] = 1
    A[idx+1, (visiting_team_idx*num_feats)] = 1
    
    # Set defensive team factor
    A[idx, (visiting_team_idx*num_feats + 1)] = 1
    A[idx+1, (home_team_idx*num_feats + 1)] = 1
    
    # Set park factor (assuming a specific column index for park factors)
    A[idx, (home_team_idx*num_feats + 2)] = 1  # Adjust this index if park factors are in a different column
    A[idx+1, (home_team_idx*num_feats + 2)] = 1

    b[idx] = row['Home Team Score'] 
    b[idx+1] =row['Visiting Team Score']
    idx += 2

In [34]:
# Solve using least squares, no weighting
x, residuals, rank, s = lstsq(A, b, rcond=None)

In [63]:
park_factors = []

# Populate the park factor scores
for team in team_data['Team']:
    park_factor = x[team_data.index[team_data['Team'] == team][0] * num_feats + 2]
    park_factors.append(park_factor)

# Create a new DataFrame with teams and their corresponding park factor scores
df_park_factors = pd.DataFrame({
    'Team': team_data['Team'],
    'ParkFactor': park_factors
})
print('Non-Weighted Park Factors!')
print(round(df_park_factors.sort_values(by='ParkFactor', ascending= False),3).head())


Non-Weighted Park Factors!
   Team             ParkFactor
8   COL   [0.7651660368586359]
27  TEX  [0.44046067284177787]
25  STL   [0.3237721316279126]
20  PHI   [0.2758237960234817]
21  PIT  [0.24068718029555058]


In [59]:
#weights = (weights / np.sum(weights)) * 100

# Add a constant term to the predictors
X = sm.add_constant(A)

# Fit the WLS model
wls_model = sm.WLS(b, X, weights=weights).fit()
beta = wls_model.params 
# Print the summary of the model
#print(wls_model.summary())

In [61]:
# Extract every third coefficient starting from the first coefficient
coefficients = beta[1:]
constant = coefficients[0]

park_factors = []

# Populate the park factor scores
for team in team_data['Team']:
    park_factor = coefficients[team_data.index[team_data['Team'] == team][0] * num_feats + 2]
    park_factors.append(park_factor)

# Create a new DataFrame with teams and their corresponding park factor scores
df_park_factors = pd.DataFrame({
    'Team': team_data['Team'],
    'ParkFactor': park_factors
})
print('Weighted Park Factors')
print(round(df_park_factors.sort_values(by='ParkFactor', ascending= False),3))


Weighted Park Factors
   Team  ParkFactor
8   COL       0.682
27  TEX       0.534
25  STL       0.309
12  LAA       0.218
20  PHI       0.191
21  PIT       0.161
16  MIN       0.155
18  NYY       0.127
15  MIL       0.101
23  SEA       0.096
0   ARI       0.072
1   ATL       0.031
5   CHW      -0.000
11  KCR      -0.016
4   CHC      -0.017
3   BOS      -0.099
29  WSN      -0.116
9   DET      -0.132
6   CIN      -0.181
28  TOR      -0.214
24  SFG      -0.236
26  TBR      -0.297
22  SDP      -0.343
7   CLE      -0.343
14  MIA      -0.397
13  LAD      -0.464
10  HOU      -0.537
17  NYM      -0.551
19  OAK      -0.560
2   BAL      -0.657
